In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [28]:
## load dataset
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [36]:
# preprocessing the data
# drop irrelevent columns
data=data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data

KeyError: "['RowNumber', 'CustomerId', 'Surname'] not found in axis"

In [30]:
# encode categorical variables
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [31]:
## onehot encode geography
from sklearn.preprocessing import OneHotEncoder                                 
onehot_encoder_geo = OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])

In [32]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [33]:
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [ ]:
## combine one hot encoder columsn with original data
data=pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [38]:
#save the encoder and sscaler
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('label_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)


In [ ]:
# dividing dataset into dependent and independent features

# all independent variables of the dataset stored in X
X=data.drop('Exited', axis=1)

# dependent variable (exited) stores in y
y=data['Exited']

# split data in training and testing sets randomly
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.2, random_state=42)

# scale these features

#transforms data to have mean = 0 and standard deviation = 1
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)

# We use transform only, not fit_transform, because we want to scale the test data using the same mean and std as the training set.
X_test=scaler.transform(X_test)

X_train

In [40]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

ANN IMPLEMENTATION

In [44]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard


In [47]:
# building our ANN model
model=Sequential([
    # dense means each neuron connected to all outputs of previous layer
    # 64 is the number of neurons in this layer
    # if your training data X_train has n features, X_train.shape[1] will be n.
    # This is only required in the first layer, as Keras needs to know the input size.    
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ## HL1

    Dense(32, activation='relu'), ## HL2
    Dense(1, activation='sigmoid') ## output layer
])

In [ ]:
model.summary()
# parameters are the combination of all the weights and bias

In [50]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
## compile the model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# optimizers defines how the model updates its weights during training using the gradients from backpropagation.
# loss defines the objective function—i.e., what the model tries to minimize.
# metrics are just for monitoring performance during training and evaluation

In [53]:
## set up the tensor board
import datetime
from tensorflow.keras.callbacks import  EarlyStopping, TensorBoard

log_dir="logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir, histogram_freq=1)


In [60]:
# setup early stopping
# During training, the model keeps improving on the training data, but after a certain point, its performance on the validation data may stop improving and start degrading 

# validation loss, patience till 10 epochs
early_stopping_callback=EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True) 



In [ ]:
# train the model
history=model.fit(
    X_train, y_train, validation_data = (X_test, y_test), epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

In [63]:
model.save('model.h5')

/Users/nikunjmahajan/Desktop/gen_ai/ANN_proj_implementation/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [64]:
# load tensorboard extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
## load the pickle file
